In [ ]:
%pip install google-colab-selenium[undetected] fake-useragent -q
print("Kütüphaneler yüklendi.")

Kütüphaneler yüklendi.


In [ ]:
import google_colab_selenium as gs
import time
import random

print("Sürücü başlatılıyor (standart modda)...") # Mesajı güncelledim
driver = None
try:
    # Standart modda sürücüyü başlat
    driver = gs.Chrome() # undetected=True argümanını kaldırdık
    print("Sürücü başarıyla başlatıldı.")

    print("Google.com'a gidiliyor...")
    driver.get("https://www.google.com")

    time.sleep(random.uniform(2, 5))

    page_title = driver.title
    print(f"Sayfa Başlığı: {page_title}")

    if "Google" in page_title:
        print("Başarılı: Google ana sayfasına erişildi.")
    else:
        print("Uyarı: Beklenen sayfa başlığı alınamadı.")

except Exception as e:
    print(f"Bir hata oluştu: {e}")

finally:
    if driver:
        print("Sürücü kapatılıyor...")
        driver.quit()
        print("Sürücü kapatıldı.")

Sürücü başlatılıyor (standart modda)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sürücü başarıyla başlatıldı.
Google.com'a gidiliyor...
Sayfa Başlığı: Google
Başarılı: Google ana sayfasına erişildi.
Sürücü kapatılıyor...
Sürücü kapatıldı.


In [ ]:
%pip install selenium webdriver-manager undetected-chromedriver fake-useragent -q
print("Kütüphaneler kontrol edildi/yüklendi.")

Kütüphaneler kontrol edildi/yüklendi.


In [ ]:
# Gerekli kütüphaneler
import time
import random
import uuid
import pandas as pd
import json # JSON işlemleri için
import re # Yıl ve kaynak temizleme için Regex
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
import undetected_chromedriver as uc

print("Undetected sürücü doğrudan başlatılıyor...")
driver = None
extracted_data = [] # Toplanan verileri tutacak liste

try:
    options = uc.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = uc.Chrome(options=options, headless=True, use_subprocess=False)
    driver.set_page_load_timeout(60) # Sayfa yükleme zaman aşımı
    driver.implicitly_wait(5) # Element bulma için genel bekleme
    print("Undetected sürücü başarıyla başlatıldı.")

    # ----- FAZ 3 Kodu (Veriyi Toplamak İçin) -----
    print("Google Scholar'a gidiliyor...")
    driver.get("https://scholar.google.com/")
    wait_time_long = random.uniform(8, 15)
    print(f"Sayfa yüklenmesi için {wait_time_long:.2f} saniye bekleniyor...")
    time.sleep(wait_time_long)
    page_title = driver.title
    print(f"Ulaşılan Sayfa Başlığı: {page_title}")
    if "Scholar" not in page_title:
        raise Exception("Google Scholar ana sayfasına erişilemedi. CAPTCHA olabilir.")
    print("Google Scholar ana sayfasına başarıyla erişildi.")

    search_keyword = "reinforcement learning applications"
    try:
        print(f"'{search_keyword}' için arama yapılıyor...")
        search_box = driver.find_element(By.ID, "gs_hdr_tsi")
        time.sleep(random.uniform(0.5, 1.0))
        search_box.send_keys(search_keyword)
        time.sleep(random.uniform(0.5, 1.0))
        search_button = driver.find_element(By.ID, "gs_hdr_tsb")
        time.sleep(random.uniform(0.5, 1.0))
        search_button.click()
        print("Arama butonuna tıklandı.")
        wait_time_search = random.uniform(10, 20)
        print(f"Arama sonuçları için {wait_time_search:.2f} saniye bekleniyor...")
        time.sleep(wait_time_search)
        results_page_title = driver.title
        print(f"Arama Sonuçları Sayfa Başlığı: {results_page_title}")
        print("Faz 3 Veri Çekme Başlıyor...")
    except NoSuchElementException as e:
        raise Exception(f"Arama kutusu/butonu bulunamadı: {e}")

    print("Sayfa aşağı kaydırılıyor...")
    for i in range(2):
        driver.execute_script(f"window.scrollBy(0, 500);")
        time.sleep(random.uniform(0.5, 1.0))

    print("Arama sonucu konteynerleri aranıyor...")
    result_containers = driver.find_elements(By.CSS_SELECTOR, "div.gs_r.gs_or.gs_scl")
    print(f"{len(result_containers)} adet sonuç bulundu (ilk sayfa).")

    if result_containers:
        print("Faz 3 Sonuçları İşleniyor...")
        for i, container in enumerate(result_containers):
            container_cid = container.get_attribute('data-cid') or str(uuid.uuid4())
            title, url, authors_line, snippet, pdf_url = None, None, None, None, None

            # Verileri çekme (try-except ile daha güvenli)
            try:
                title_element = container.find_element(By.CSS_SELECTOR, "h3.gs_rt a")
                title = title_element.text
                url = title_element.get_attribute('href')
            except NoSuchElementException: pass # Hata yoksa devam et
            try:
                authors_element = container.find_element(By.CSS_SELECTOR, "div.gs_a")
                authors_line = authors_element.text
            except NoSuchElementException: pass
            try:
                snippet_element = container.find_element(By.CSS_SELECTOR, "div.gs_rs")
                snippet = snippet_element.text.strip()
            except NoSuchElementException: pass
            try:
                pdf_section = None
                try: pdf_section = container.find_element(By.CSS_SELECTOR, "div.gs_ggs")
                except NoSuchElementException:
                    try: pdf_section = container.find_element(By.CSS_SELECTOR, "div.gs_fl")
                    except NoSuchElementException: pass

                if pdf_section:
                     pdf_links = pdf_section.find_elements(By.TAG_NAME, "a")
                     for link in pdf_links:
                         href = link.get_attribute('href')
                         if href and (href.lower().endswith('.pdf') or '[pdf]' in link.text.lower()):
                             pdf_url = href
                             break
            except Exception as pdf_e: print(f"  Uyarı: Sonuç {i+1} için PDF linki aranırken hata: {pdf_e}")

            extracted_data.append({
                'id': container_cid, 'source': 'google_scholar', 'title': title, 'abstract': snippet,
                'authors': authors_line, 'published_date': None, 'publish_year': None,
                'updated_date': None, 'url': url, 'pdf_url': pdf_url, 'doi': None })
            time.sleep(random.uniform(0.5, 1.5))
        print(f"Faz 3: {len(extracted_data)} sonuç için temel veriler toplandı.")
    else:
        print("Faz 3'te hiç sonuç konteyneri bulunamadı.")
    # ----- FAZ 3 Kodu Bitiş -----


    # ----- FAZ 5 BAŞLANGIÇ (Veri İyileştirme ve Kaydetme) -----
    print("\nFaz 5 Başlıyor: Veri iyileştirme ve dosyalara kaydetme...")

    if not extracted_data:
        print("Kaydedilecek veri bulunamadı.")
    else:
        print("Veri iyileştiriliyor (Yıl ve Yazar Temizleme)...")
        processed_count = 0
        current_system_year = pd.Timestamp.now().year # Sistem yılı kontrol için
        for item in extracted_data:
            authors_str = item.get('authors')
            year = None
            cleaned_authors = authors_str

            if isinstance(authors_str, str) and authors_str:
                # Regex ile metin içindeki 4 haneli sayıları bul
                possible_years = []
                for yr_str in re.findall(r'(\b\d{4}\b)', authors_str):
                    yr = int(yr_str)
                    # Makul bir yıl aralığında mı kontrol et (örn: 1900 - sistem yılı + 2)
                    if 1900 <= yr <= current_system_year + 2:
                        possible_years.append(yr)
                if possible_years:
                    # Genellikle en sondaki yıl yayın yılıdır
                    year = possible_years[-1]

                # Yazar kısmını temizleme: ' - ' ile ayrılan kısımlardan sonuncuyu atmayı dene
                parts = authors_str.split(' - ')
                if len(parts) > 1:
                     # Eğer yıl bulduysak ve sondan ikinci kısımda ise daha güvenli
                     if year and str(year) in parts[-2]:
                          cleaned_authors = ' - '.join(parts[:-1]).strip()
                     else: # Değilse, yine de son kısmı kaynak varsayarak atalım
                          cleaned_authors = ' - '.join(parts[:-1]).strip()
                # Eğer hiç ' - ' yoksa orijinal kalsın
                elif len(parts) == 1:
                     cleaned_authors = authors_str # Zaten sadece yazar(lar)

                # Temizlenmiş yazar listesinde sonda kalmış olabilecek virgülü kaldır
                if cleaned_authors and cleaned_authors.endswith(','):
                    cleaned_authors = cleaned_authors[:-1].strip()

            # Güncellenmiş değerleri sözlüğe yaz
            item['publish_year'] = year
            item['authors'] = cleaned_authors if cleaned_authors else authors_str # Eğer temizleme sonucu boş kalırsa orijinali kullan
            processed_count += 1
        print(f"{processed_count} adet kaydın yıl ve yazar bilgisi işlendi.")

        # Pandas DataFrame oluştur
        df = pd.DataFrame(extracted_data)

        # İstenen Sütun Sırası
        final_columns = []
        available_columns = df.columns.tolist()
        desired_columns = [
            'id', 'source', 'title', 'abstract', 'authors',
            'published_date', 'publish_year', 'updated_date',
            'url', 'pdf_url', 'doi'
        ]
        for col in desired_columns:
            if col in available_columns:
                final_columns.append(col)
        df = df[final_columns] # Var olan sütunları sırala

        # CSV olarak kaydet
        csv_filename = 'google_scholar_results.csv'
        try:
            df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
            print(f"Veriler başarıyla '{csv_filename}' dosyasına kaydedildi.")
        except Exception as e:
            print(f"CSV dosyasına kaydederken HATA oluştu: {e}")

        # JSON olarak kaydet
        json_filename = 'google_scholar_results.json'
        try:
            # NaN/None değerlerini JSON için null yapalım
            df_for_json = df.where(pd.notnull(df), None)
            json_data = df_for_json.to_dict(orient='records')
            with open(json_filename, 'w', encoding='utf-8') as f:
                json.dump(json_data, f, ensure_ascii=False, indent=4)
            print(f"Veriler başarıyla '{json_filename}' dosyasına kaydedildi.")
        except Exception as e:
            print(f"JSON dosyasına kaydederken HATA oluştu: {e}")

        # Sonucu tekrar gösterelim (iyileştirilmiş haliyle)
        print("\nİyileştirilmiş ilk 5 sonuç:")
        display_columns = ['id', 'title', 'authors', 'publish_year', 'abstract', 'url', 'pdf_url', 'doi']
        print(df.head()[display_columns].fillna('').to_markdown(index=False))

    # ----- FAZ 5 BİTİŞ -----

except Exception as e:
    print(f"Genel bir HATA oluştu: {e}")
    try:
        if driver:
             driver.save_screenshot("final_error.png")
             print("Hata ekran görüntüsü 'final_error.png' olarak kaydedildi.")
    except Exception as ss_e:
        print(f"Ekran görüntüsü alınamadı: {ss_e}")

finally:
    # Sürücüyü her durumda kapat
    if driver:
        print("İşlem bitti, sürücü kapatılıyor...")
        time.sleep(random.uniform(2, 5)) # Kısa bir son bekleme
        driver.quit()
        print("Sürücü kapatıldı.")

Undetected sürücü doğrudan başlatılıyor...
Undetected sürücü başarıyla başlatıldı.
Google Scholar'a gidiliyor...
Sayfa yüklenmesi için 11.18 saniye bekleniyor...
Ulaşılan Sayfa Başlığı: Google Scholar
Google Scholar ana sayfasına başarıyla erişildi.
'reinforcement learning applications' için arama yapılıyor...
Arama butonuna tıklandı.
Arama sonuçları için 13.61 saniye bekleniyor...
Arama Sonuçları Sayfa Başlığı: 403
Faz 3 Veri Çekme Başlıyor...
Sayfa aşağı kaydırılıyor...
Arama sonucu konteynerleri aranıyor...
0 adet sonuç bulundu (ilk sayfa).
Faz 3'te hiç sonuç konteyneri bulunamadı.

Faz 5 Başlıyor: Veri iyileştirme ve dosyalara kaydetme...
Kaydedilecek veri bulunamadı.
İşlem bitti, sürücü kapatılıyor...
Sürücü kapatıldı.
